In [3]:
using Plots
plotlyjs()

Plots.PlotlyJSBackend()

In [2]:
function Rotmat(θ::Float64)
    R=Array{Float64,2}(2,2)
    R[1,1]=cos(θ)
    R[2,2]=cos(θ)
    R[1,2]=-sin(θ)
    R[2,1]=sin(θ)
    return R
end

Rotmat (generic function with 1 method)

\begin{equation}
R_1=\begin{pmatrix}
\cos \omega + \nu & -sin \omega + \nu & 0 \\
sin \omega + \nu  & \cos \omega + \nu & 0 \\
0 & 0 & 1 \\
\end{pmatrix}
\end{equation}

\begin{equation}
R_2 = \begin{pmatrix}
\cos i & 0 &- \sin i \\
0 & 1 & 0 \\
\sin i & 0 & \cos i \\
\end{pmatrix}
\end{equation}

\begin{equation}
R_3= \begin{pmatrix}
\cos \Omega & -\sin \Omega & 0 \\
\sin \Omega & \cos \Omega & 0 \\
0 & 0 & 1 \\
\end{pmatrix}
\end{equation}


![](Images/Kepler/Orbit1.svg)
By Lasunncty at the English language Wikipedia, CC BY-SA 3.0, https://commons.wikimedia.org/w/index.php?curid=8971052\

![](Images/Kepler/ellipse.svg)

In [ ]:
type coordCart
    x::Float64
    y::Float64
    z::Float64
    
    vx::Float64
    vy::Float64
    vz::Float64
end

In [ ]:
type coordOrb
    a::Float64 # Semimajor Axis
    e::Float64  # eccentricity
    i::Float64  # inclination
    
    ω::Float64 # \omega Argument of periapsis
    Ω::Float64 # \Omega Longitude of ascending node
    
    ν::Float64 # \nu True Anomaly
end

In [ ]:
type coordDel
    L::Float64
    G::Float64
    H::Float64
    
    l::Float64
    g::Float64
    h::Float64
end

Reduced Mass of System
\begin{equation}
\frac{1}{\mu} = \frac{1}{M_1} + \frac{1}{M_2}
\end{equation}

\begin{equation}
r = \sqrt{x^2 + y^2 + z^2 }
\end{equation}

\begin{equation}
v= \sqrt{v_x^2+v_y^2+v_z^2}
\end{equation}

\begin{equation}
\vec{h}= \vec{x} \times \vec{v}
\qquad 
h = |\vec{h}|
\end{equation}

\begin{equation}
E = \frac{1}{2} v^2 - \frac{\mu}{r}
\end{equation}

\begin{equation}
a = - \frac{\mu}{2 E}
\end{equation}

\begin{equation}
e = \sqrt{ \left( 1 -\frac{h^2}{a \mu} \right) }
\end{equation}

\begin{equation}
i = \text{acos} \frac{h_z}{h}
\end{equation}

\begin{equation}
\Omega = \text{atan}^2- \frac{h_x}{h_y}
\end{equation}

\begin{equation}
\omega = \text{atan}^2 \frac{z /\sin i }{x * \cos \Omega + y * \sin \Omega }
\end{equation}
\begin{equation}
\nu = \text{acos} \left( \frac{ a \left( 1 -e^2 \right) -r}{e r} \right)
\end{equation}

http://ccar.colorado.edu/asen5070/handouts/cart2kep2002.pdf

In [ ]:
function CarttoOrb(C::coordCart,μ::Float64)
    r=sqrt(C.x^2+C.y^2+C.z^2)
    v=sqrt(C.vx^2+C.vy^2+C.vz^2)
    hvec=cross([C.x,C.y,C.z],[C.vx,C.vy,C.vz])
    h=norm(hvec)
    
    E=v^2/2-μ/r
    
    a=-μ/(2*E)
    e=sqrt(1-(h^2)/(a*μ))
    
    i=acos(hvec[3]/h)
    
    Ω=atan2(hvec[1],-hvec[2])
    
    ων=atan2(C.z/sin(i),C.x*cos(Ω)+C.y*sin(Ω))
    ν=acos((a*(1-e^2)-r)/(e*r))
    
    
   return coordOrb(a,e,i,Ω,ω,ν)
end

In [ ]:
function OrbtoCart(O::coordOrb,μ::Float64)
    r=(O.a*(1-O.e^2))/(1+O.e*cos(O.ν))
    h=sqrt(μ*O.a*(1-O.e^2))
    v=h/r
    
    R1=eye(Float64,3)
    R1[[1,2],[1,2]]=Rotmat(O.ω+O.ν)
    
    R2=eye(Float64,3)
    R2[[1,3],[1,3]]=Rotmat(O.i)
    
    R3=eye(Float64,3)
    R3[[1,2],[1,2]]=Rotmat(O.Ω)
    
    pos=r*R3*R2*R1*[1,0,0]
    vel=v*R3*R2*R1*[0,1,0]
    
   return coordCart(pos[1],pos[2],pos[3],vel[1],vel[2],vel[3])
end

In [ ]:
function OrbtoDel(O::coordOrb,μ::Float64)
    L=√(μ*O.a)
    G=L*√(1-O.e^2)
    H=cos(O.i)*G
    return coordDel(L,G,H,O.ν,O.ω,O.Ω)
end

In [ ]:
νi=linspace(0,2π,100)
orbit=Array{coordCart}(100)

for ii in 1:100
    orbit[ii] =OrbtoCart(coordOrb(1,.2,0,π/2,0,νi[ii]),1.) 
end

In [ ]:
plot([or.x for or in orbit],[or.y for or in orbit])
hline!([-1,1])
vline!([-1,1])